In [1]:
import requests
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

In [2]:
# Base API URL
BASE_URL = "https://fantasy.premierleague.com/api"

# Helper function to fetch JSON data
def fetch_data(url):
    return requests.get(url).json()

# Fetch data for the 2024-2025 season
bootstrap_static_2025 = fetch_data(f"{BASE_URL}/bootstrap-static/")
players_2025 = pd.DataFrame(bootstrap_static_2025['elements'])
teams_2025 = pd.DataFrame(bootstrap_static_2025['teams'])
positions_2025 = pd.DataFrame(bootstrap_static_2025['element_types'])

# Map team and position IDs to readable names
team_map_2025 = teams_2025.set_index('id')['name'].to_dict()
position_map_2025 = positions_2025.set_index('id')['singular_name_short'].to_dict()

# Add readable fields for players
players_2025['team'] = players_2025['team'].map(team_map_2025)
players_2025['position'] = players_2025['element_type'].map(position_map_2025)
players_2025['full_name'] = players_2025['first_name'] + " " + players_2025['second_name']

# Fetch current season (2024-2025) performance data
gameweek_data_2025 = []
for player_id in players_2025['id']:
    player_data = fetch_data(f"{BASE_URL}/element-summary/{player_id}/")
    history = player_data.get('history', [])  # Use 'history' for current season data

    if history:
        # Calculate total points, matches played, and other metrics from current gameweek data
        total_points = sum(week['total_points'] for week in history)
        minutes_played = sum(week['minutes'] for week in history)
        matches_played = minutes_played // 90  # Estimate games played
        
        gameweek_data_2025.append({
            'id': player_id,
            'full_name': players_2025.loc[players_2025['id'] == player_id, 'full_name'].values[0],
            'team': players_2025.loc[players_2025['id'] == player_id, 'team'].values[0],
            'position': players_2025.loc[players_2025['id'] == player_id, 'position'].values[0],
            'total_points': total_points,
            'matches_played': matches_played,
            'value': players_2025.loc[players_2025['id'] == player_id, 'now_cost'].values[0] / 10  # Value in millions
        })

# Convert to DataFrame
df_2025 = pd.DataFrame(gameweek_data_2025)

# Calculate PPM (Points Per Million) and Points Per Match
df_2025['points_per_match'] = df_2025['total_points'] / df_2025['matches_played']
df_2025['ppm'] = df_2025['points_per_match'] / df_2025['value']

df_2025.to_csv('FPL 2024-2025 Player Performance.csv', index=False)
df_2025.head()

,id,full_name,team,position,total_points,matches_played,value,points_per_match,ppm
0,1,Fábio Ferreira Vieira,Arsenal,MID,0,0,5.4,NaN,NaN
1,2,Gabriel Fernando de Jesus,Arsenal,FWD,28,4,6.8,7.000000,1.029412
2,3,Gabriel dos Santos Magalhães,Arsenal,DEF,79,15,6.3,5.266667,0.835979
3,4,Kai Havertz,Arsenal,FWD,74,16,7.9,4.625000,0.585443
4,5,Karl Hein,Arsenal,GKP,0,0,4.0,NaN,NaN


In [23]:
# Calculate the total number of matches played by any team so far
# Assuming the maximum matches played by any player is the total matches played so far
total_games_so_far = df_2025['matches_played'].max()  # Use the maximum matches played

# Filter the dataframe for players who played at least 60% of the games
filtered_df = df_2025[df_2025['matches_played'] >= 0.7 * total_games_so_far]

# Extract the top 10 players for each position based on points per match
top_10_per_position = (
    filtered_df.sort_values(by='points_per_match', ascending=False)
    .groupby('position')
    .head(10)
)

# Keep only the relevant columns and order the output
top_10_per_position = top_10_per_position[['full_name', 'position', 'team', 'points_per_match', 'matches_played', 'value']]
top_10_per_position = top_10_per_position.sort_values(by=['position', 'points_per_match'], ascending=[True, False])
top_10_per_position

,full_name,position,team,points_per_match,matches_played,value
2,Gabriel dos Santos Magalhães,DEF,Arsenal,5.266667,15,6.3
391,Trent Alexander-Arnold,DEF,Liverpool,5.230769,13,7.1
284,Ashley Young,DEF,Everton,5.153846,13,4.7
524,Ola Aina,DEF,Nott'm Forest,4.764706,17,5.1
5,Jurriën Timber,DEF,Arsenal,4.500000,14,5.7
14,William Saliba,DEF,Arsenal,4.375000,16,6.3
498,Lewis Hall,DEF,Newcastle,4.333333,15,4.7
552,Nikola Milenković,DEF,Nott'm Forest,4.176471,17,4.7
433,Joško Gvardiol,DEF,Man City,4.125000,16,6.0
417,Virgil van Dijk,DEF,Liverpool,4.000000,17,6.5


In [24]:
# Create the optimization problem
prob = LpProblem("FPL_Team_Selection", LpMaximize)

# Create a binary variable for each player (1 if selected, 0 otherwise)
player_vars = {i: LpVariable(f"player_{i}", cat='Binary') for i in top_10_per_position.index}

# Objective function: Maximize total points per match (PPM)
prob += lpSum(player_vars[i] * top_10_per_position.loc[i, 'points_per_match'] for i in top_10_per_position.index)

# Constraints

# Total budget <= 100
prob += lpSum(player_vars[i] * top_10_per_position.loc[i, 'value'] for i in top_10_per_position.index) <= 100

# Total players = 15
prob += lpSum(player_vars[i] for i in top_10_per_position.index) == 15

# Position constraints
prob += lpSum(player_vars[i] for i in top_10_per_position.index if top_10_per_position.loc[i, 'position'] == 'GKP') == 2
prob += lpSum(player_vars[i] for i in top_10_per_position.index if top_10_per_position.loc[i, 'position'] == 'DEF') == 5
prob += lpSum(player_vars[i] for i in top_10_per_position.index if top_10_per_position.loc[i, 'position'] == 'MID') == 5
prob += lpSum(player_vars[i] for i in top_10_per_position.index if top_10_per_position.loc[i, 'position'] == 'FWD') == 3

# Team constraints (max 3 players from any one team)
teams = top_10_per_position['team'].unique()
for team in teams:
    prob += lpSum(player_vars[i] for i in top_10_per_position.index if top_10_per_position.loc[i, 'team'] == team) <= 3

# Solve the problem
prob.solve()

# Extract the selected players
selected_players = top_10_per_position.loc[[i for i in top_10_per_position.index if player_vars[i].value() == 1]]

# Order by position and points per match
selected_players = selected_players.sort_values(by=['position', 'points_per_match'], ascending=[True, False])

selected_players

,full_name,position,team,points_per_match,matches_played,value
2,Gabriel dos Santos Magalhães,DEF,Arsenal,5.266667,15,6.3
284,Ashley Young,DEF,Everton,5.153846,13,4.7
524,Ola Aina,DEF,Nott'm Forest,4.764706,17,5.1
5,Jurriën Timber,DEF,Arsenal,4.500000,14,5.7
498,Lewis Hall,DEF,Newcastle,4.333333,15,4.7
674,Matheus Santos Carneiro Da Cunha,FWD,Wolves,6.875000,16,7.1
548,Chris Wood,FWD,Nott'm Forest,6.266667,15,6.6
100,Francisco Evanilson de Lima Barbosa,FWD,Bournemouth,5.230769,13,6.0
281,Jordan Pickford,GKP,Everton,4.823529,17,4.9
545,Matz Sels,GKP,Nott'm Forest,4.388889,18,4.8


In [25]:
print(selected_players['points_per_match'].sum())

88.41618903971845


In [26]:
print(selected_players['value'].sum())

99.89999999999999
